In [1]:
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination

c:\Users\thiel\.conda\envs\project3-1\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Sample data with columns: Weather, Mood, Run
data = pd.DataFrame({
    'Weather': [1, 0, 1, 0, 1, 0, 1, 0],
    'Mood': [1, 0, 1, 0, 1, 0, 1, 0],
    'Run': [1, 0, 1, 0, 0, 0, 1, 1]
})

In [3]:
# Create a Bayesian Network model
model = BayesianNetwork([('Weather', 'Run'), ('Mood', 'Run')])

# Fit the model to the data using Maximum Likelihood Estimators
model.fit(data, estimator=MaximumLikelihoodEstimator)

# Iterate through the nodes and print the CPDs
for node in model.nodes():
    cpd = model.get_cpds(node)
    print(f"CPD for {node}:")
    print(cpd)
    print()

CPD for Weather:
+------------+-----+
| Weather(0) | 0.5 |
+------------+-----+
| Weather(1) | 0.5 |
+------------+-----+

CPD for Run:
+---------+------------+------------+------------+------------+
| Mood    | Mood(0)    | Mood(0)    | Mood(1)    | Mood(1)    |
+---------+------------+------------+------------+------------+
| Weather | Weather(0) | Weather(1) | Weather(0) | Weather(1) |
+---------+------------+------------+------------+------------+
| Run(0)  | 0.75       | 0.5        | 0.5        | 0.25       |
+---------+------------+------------+------------+------------+
| Run(1)  | 0.25       | 0.5        | 0.5        | 0.75       |
+---------+------------+------------+------------+------------+

CPD for Mood:
+---------+-----+
| Mood(0) | 0.5 |
+---------+-----+
| Mood(1) | 0.5 |
+---------+-----+



In [4]:
# Define the evidence for which you want to make predictions
evidence = {'Weather': 1, 'Mood': 1}

# Create an inference object
inference = VariableElimination(model)

# Calculate the probability of 'Run' given the evidence
predicted_run_probability = inference.query(variables=['Run'], evidence=evidence)

print(f'evidence: {evidence}')
print(predicted_run_probability)


evidence: {'Weather': 1, 'Mood': 1}
+--------+------------+
| Run    |   phi(Run) |
+========+============+
| Run(0) |     0.2500 |
+--------+------------+
| Run(1) |     0.7500 |
+--------+------------+
